In [1]:
import os, random, shutil
from scipy.io import wavfile
from shutil import copyfile
from sphfile import SPHFile
from WaveNetClassifier import WaveNetClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import h5py

Using TensorFlow backend.


In [2]:
# Each person has 10 audio samples.

# Normalize the data before saving them to arrays.
nb_bits = 16
max_nb_bit = float(2 ** (nb_bits - 1))

def process_audio(base_dir, audio_data, user_ids, test_audio_data, test_user_ids, no_users):
    min_files = 0
    for i, user_id in enumerate(os.listdir(base_dir)):
        if i == no_users:
            return
        current_dir = os.path.join(base_dir, user_id)
        for i, audio_file in enumerate(os.listdir(current_dir)):
            no_samples = len(os.listdir(current_dir))
            file_path = os.path.join(current_dir, audio_file)
            fs, data = wavfile.read(file_path)
            if i < no_samples - 1:
                user_ids.append(user_id)
                audio_data.append(data / (max_nb_bit + 1.0))
            else:
                test_user_ids.append(user_id)
                test_audio_data.append(data / (max_nb_bit + 1.0))

In [3]:
def process_audio_1_sample(base_dir, audio_data, user_ids, test_audio_data, test_user_ids, no_users):
    min_files = 0
    for i, user_id in enumerate(os.listdir(base_dir)):
        if i == no_users:
            return
        current_dir = os.path.join(base_dir, user_id)
        for i, audio_file in enumerate(os.listdir(current_dir)):
            file_path = os.path.join(current_dir, audio_file)
            fs, data = wavfile.read(file_path)
            if i == 1:
                user_ids.append(user_id)
                audio_data.append(data / (max_nb_bit + 1.0))
            elif i == 2:
                test_user_ids.append(user_id)
                test_audio_data.append(data / (max_nb_bit + 1.0))
            else:
                continue

In [4]:
def process_audio_2_samples(base_dir, audio_data, user_ids, test_audio_data, test_user_ids, no_users):
    min_files = 0
    for i, user_id in enumerate(os.listdir(base_dir)):
        if i == no_users:
            return
        current_dir = os.path.join(base_dir, user_id)
        for i, audio_file in enumerate(os.listdir(current_dir)):
            file_path = os.path.join(current_dir, audio_file)
            fs, data = wavfile.read(file_path)
            if i < 2:
                user_ids.append(user_id)
                audio_data.append(data / (max_nb_bit + 1.0))
            elif i == 2:
                test_user_ids.append(user_id)
                test_audio_data.append(data / (max_nb_bit + 1.0))
            else:
                continue

In [5]:
def unify_audio(audio_data, test_audio_data):
    min_length_audio = min(audio_data, key = len)
    min_length_test_audio = min(test_audio_data, key = len)
    min_length = min(len(min_length_audio), len(min_length_test_audio))
    for i, sample in enumerate(audio_data):
        audio_data[i] = sample[0 : min_length]
    for i, sample in enumerate(test_audio_data):
        test_audio_data[i] = sample[0 : min_length]

In [74]:
# Remove samples less than x length.

base_dir = 'TRAIN_WAV_NO_DR_64k'

for user_id in os.listdir(base_dir):
    current_dir = os.path.join(base_dir, user_id)
    for audio_file in os.listdir(current_dir):
        file_path = os.path.join(current_dir, audio_file)
        fs, data = wavfile.read(file_path)
        if len(data) < 64000:
            if(os.path.exists(file_path)):
                os.remove(file_path)

In [84]:
# Delete users with 0 or 1 samples.

base_dir = 'TRAIN_WAV_NO_DR'

for user_id in os.listdir(base_dir):
    current_dir = os.path.join(base_dir, user_id)
    if len(os.listdir(current_dir)) < 2:
        shutil.rmtree(current_dir)

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

In [99]:
# List no of samples for each user.

base_dir = 'TRAIN_WAV_NO_DR_48k'

lens = []

for user_id in os.listdir(base_dir):
    current_dir = os.path.join(base_dir, user_id)
    lens.append(len(os.listdir(current_dir)))
                
sorted(lens)

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,


In [11]:
audio_data = []
user_ids = []
test_audio_data = []
test_user_ids = []
base_dir = 'TRAIN_WAV_NO_DR'
no_users = 6
max_no_users = len(os.listdir(base_dir))

process_audio(base_dir, audio_data, user_ids, test_audio_data, test_user_ids, 100)

display(len(audio_data))
display(len(user_ids))
display(len(test_audio_data))
display(len(test_user_ids))

900

900

100

100

In [12]:
unify_audio(audio_data, test_audio_data)

display(len(audio_data[4]))
display(len(test_audio_data[2]))

18330

18330

In [13]:
# Create a vector for each user id. Categorical -> one hot encoding
# The list has to be converted the numpy array and transposed first.

X_train = np.array(audio_data)

onehot_encoder = OneHotEncoder(sparse = False, categories = 'auto')
Y_train = onehot_encoder.fit_transform(np.array(user_ids).reshape(-1, 1))

print('X_train shape:')
display(X_train.shape)

print('Y_train shape:')
display(Y_train.shape)

X_train shape:


(900, 18330)

Y_train shape:


(900, 100)

In [123]:
wnc = WaveNetClassifier((32052,), (6,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 32052)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 32052, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 32052, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 32052, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [127]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 10, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
51/51 [==============================] - 11s 223ms/step - loss: 1.8128 - acc: 0.1373

Epoch 00001: loss improved from inf to 1.81276, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
51/51 [==============================] - 5s 96ms/step - loss: 1.7931 - acc: 0.1765

Epoch 00002: loss improved from 1.81276 to 1.79309, saving model to results3/saved_wavenet_clasifier.h5
Epoch 3/200
51/51 [==============================] - 5s 95ms/step - loss: 1.7918 - acc: 0.1961

Epoch 00003: loss improved from 1.79309 to 1.79179, saving model to results3/saved_wavenet_clasifier.h5
Epoch 4/200
51/51 [==============================] - 5s 96ms/step - loss: 1.7918 - acc: 0.1765

Epoch 00004: loss did not improve from 1.79179
Epoch 5/200
51/51 [==============================] - 5s 95ms/step - loss: 1.7918 - acc: 0.1569

Epoch 00005: loss improved from 1.79179 to 1.79178, saving model to results3/saved_wavenet_clasifier.h5
Epoch 6/200
51/51 [==============================] - 5s 96m

51/51 [==============================] - 5s 98ms/step - loss: 1.7907 - acc: 0.1765

Epoch 00046: loss improved from 1.79078 to 1.79070, saving model to results3/saved_wavenet_clasifier.h5
Epoch 47/200
51/51 [==============================] - 5s 97ms/step - loss: 1.7908 - acc: 0.1765

Epoch 00047: loss did not improve from 1.79070
Epoch 48/200
51/51 [==============================] - 5s 96ms/step - loss: 1.7908 - acc: 0.1765

Epoch 00048: loss did not improve from 1.79070
Epoch 49/200
51/51 [==============================] - 5s 96ms/step - loss: 1.7906 - acc: 0.1765

Epoch 00049: loss improved from 1.79070 to 1.79065, saving model to results3/saved_wavenet_clasifier.h5
Epoch 50/200
51/51 [==============================] - 5s 96ms/step - loss: 1.7906 - acc: 0.1765

Epoch 00050: loss improved from 1.79065 to 1.79056, saving model to results3/saved_wavenet_clasifier.h5
Epoch 51/200
51/51 [==============================] - 5s 96ms/step - loss: 1.7906 - acc: 0.1765

Epoch 00051: loss did not

51/51 [==============================] - 5s 97ms/step - loss: 1.7903 - acc: 0.1765

Epoch 00097: loss improved from 1.79026 to 1.79026, saving model to results3/saved_wavenet_clasifier.h5
Epoch 98/200
51/51 [==============================] - 5s 97ms/step - loss: 1.7903 - acc: 0.1765

Epoch 00098: loss did not improve from 1.79026
Epoch 99/200
51/51 [==============================] - 5s 97ms/step - loss: 1.7903 - acc: 0.1765

Epoch 00099: loss did not improve from 1.79026
Epoch 100/200
51/51 [==============================] - 5s 97ms/step - loss: 1.7903 - acc: 0.1765

Epoch 00100: loss improved from 1.79026 to 1.79026, saving model to results3/saved_wavenet_clasifier.h5
Epoch 101/200
51/51 [==============================] - 5s 96ms/step - loss: 1.7903 - acc: 0.1765

Epoch 00101: loss did not improve from 1.79026
Epoch 102/200
20/51 [==========>...................] - ETA: 2s - loss: 1.7936 - acc: 0.2000

KeyboardInterrupt: 

In [131]:
wnc = WaveNetClassifier((48333,), (6,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 48333)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 48333, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 48333, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 48333, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [133]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 7, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
28/28 [==============================] - 10s 358ms/step - loss: 1.8007 - acc: 0.1071

Epoch 00001: loss improved from inf to 1.80071, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
28/28 [==============================] - 4s 158ms/step - loss: 1.7915 - acc: 0.2500

Epoch 00002: loss improved from 1.80071 to 1.79149, saving model to results3/saved_wavenet_clasifier.h5
Epoch 3/200
28/28 [==============================] - 4s 156ms/step - loss: 1.7911 - acc: 0.2500

Epoch 00003: loss improved from 1.79149 to 1.79109, saving model to results3/saved_wavenet_clasifier.h5
Epoch 4/200
28/28 [==============================] - 4s 157ms/step - loss: 1.7908 - acc: 0.2500

Epoch 00004: loss improved from 1.79109 to 1.79079, saving model to results3/saved_wavenet_clasifier.h5
Epoch 5/200
28/28 [==============================] - 4s 157ms/step - loss: 1.7905 - acc: 0.2500

Epoch 00005: loss improved from 1.79079 to 1.79046, saving model to results3/saved_wavenet_clasifier.h

Epoch 42/200
28/28 [==============================] - 4s 157ms/step - loss: 1.7804 - acc: 0.2500

Epoch 00042: loss improved from 1.78056 to 1.78035, saving model to results3/saved_wavenet_clasifier.h5
Epoch 43/200
28/28 [==============================] - 4s 157ms/step - loss: 1.7802 - acc: 0.2500

Epoch 00043: loss improved from 1.78035 to 1.78016, saving model to results3/saved_wavenet_clasifier.h5
Epoch 44/200
28/28 [==============================] - 4s 157ms/step - loss: 1.7799 - acc: 0.2500

Epoch 00044: loss improved from 1.78016 to 1.77994, saving model to results3/saved_wavenet_clasifier.h5
Epoch 45/200
28/28 [==============================] - 4s 158ms/step - loss: 1.7796 - acc: 0.2500

Epoch 00045: loss improved from 1.77994 to 1.77963, saving model to results3/saved_wavenet_clasifier.h5
Epoch 46/200
28/28 [==============================] - 4s 158ms/step - loss: 1.7794 - acc: 0.2500

Epoch 00046: loss improved from 1.77963 to 1.77944, saving model to results3/saved_wavenet_cla

KeyboardInterrupt: 

In [137]:
wnc = WaveNetClassifier((67072,), (6,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 67072)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 67072, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 67072, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 67072, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [138]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 5, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
11/11 [==============================] - 9s 830ms/step - loss: 1.7986 - acc: 0.2727

Epoch 00001: loss improved from inf to 1.79864, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
11/11 [==============================] - 2s 224ms/step - loss: 1.7763 - acc: 0.2727

Epoch 00002: loss improved from 1.79864 to 1.77635, saving model to results3/saved_wavenet_clasifier.h5
Epoch 3/200
11/11 [==============================] - 2s 220ms/step - loss: 1.8586 - acc: 0.2727

Epoch 00003: loss did not improve from 1.77635
Epoch 4/200
11/11 [==============================] - 2s 220ms/step - loss: 1.7382 - acc: 0.2727

Epoch 00004: loss improved from 1.77635 to 1.73823, saving model to results3/saved_wavenet_clasifier.h5
Epoch 5/200
11/11 [==============================] - 2s 221ms/step - loss: 1.7764 - acc: 0.2727

Epoch 00005: loss did not improve from 1.73823
Epoch 6/200
11/11 [==============================] - 2s 221ms/step - loss: 1.7734 - acc: 0.2727

Epoch 00006: los

11/11 [==============================] - 2s 221ms/step - loss: 1.7614 - acc: 0.2727

Epoch 00056: loss did not improve from 1.73823
Epoch 57/200
11/11 [==============================] - 2s 221ms/step - loss: 1.7609 - acc: 0.2727

Epoch 00057: loss did not improve from 1.73823
Epoch 58/200
11/11 [==============================] - 2s 221ms/step - loss: 1.7605 - acc: 0.2727

Epoch 00058: loss did not improve from 1.73823
Epoch 59/200
11/11 [==============================] - 2s 221ms/step - loss: 1.7596 - acc: 0.2727

Epoch 00059: loss did not improve from 1.73823
Epoch 60/200
11/11 [==============================] - 2s 221ms/step - loss: 1.7590 - acc: 0.2727

Epoch 00060: loss did not improve from 1.73823
Epoch 61/200
11/11 [==============================] - 2s 222ms/step - loss: 1.7584 - acc: 0.2727

Epoch 00061: loss did not improve from 1.73823
Epoch 62/200
11/11 [==============================] - 2s 221ms/step - loss: 1.7577 - acc: 0.2727

Epoch 00062: loss did not improve from 1.7382

11/11 [==============================] - 2s 222ms/step - loss: 1.7338 - acc: 0.2727

Epoch 00112: loss did not improve from 1.67241
Epoch 113/200
11/11 [==============================] - 2s 222ms/step - loss: 1.7332 - acc: 0.2727

Epoch 00113: loss did not improve from 1.67241
Epoch 114/200
11/11 [==============================] - 2s 222ms/step - loss: 1.7327 - acc: 0.2727

Epoch 00114: loss did not improve from 1.67241
Epoch 115/200
11/11 [==============================] - 2s 222ms/step - loss: 1.7322 - acc: 0.2727

Epoch 00115: loss did not improve from 1.67241
Epoch 116/200
11/11 [==============================] - 2s 221ms/step - loss: 1.7318 - acc: 0.2727

Epoch 00116: loss did not improve from 1.67241
Epoch 117/200
11/11 [==============================] - 2s 221ms/step - loss: 1.7312 - acc: 0.2727

Epoch 00117: loss did not improve from 1.67241
Epoch 118/200
11/11 [==============================] - 2s 222ms/step - loss: 1.7307 - acc: 0.2727

Epoch 00118: loss did not improve from 

11/11 [==============================] - 2s 222ms/step - loss: 1.7017 - acc: 0.2727

Epoch 00168: loss did not improve from 1.67241
Epoch 169/200
11/11 [==============================] - 2s 222ms/step - loss: 1.7015 - acc: 0.2727

Epoch 00169: loss did not improve from 1.67241
Epoch 170/200
11/11 [==============================] - 2s 222ms/step - loss: 1.7006 - acc: 0.2727

Epoch 00170: loss did not improve from 1.67241
Epoch 171/200
11/11 [==============================] - 2s 221ms/step - loss: 1.6997 - acc: 0.2727

Epoch 00171: loss did not improve from 1.67241
Epoch 172/200
11/11 [==============================] - 2s 222ms/step - loss: 1.6992 - acc: 0.2727

Epoch 00172: loss did not improve from 1.67241
Epoch 173/200
11/11 [==============================] - 2s 222ms/step - loss: 1.6988 - acc: 0.2727

Epoch 00173: loss did not improve from 1.67241
Epoch 174/200
11/11 [==============================] - 2s 221ms/step - loss: 1.6979 - acc: 0.2727

Epoch 00174: loss did not improve from 

In [153]:
# 1 sample for 1 user
# no_users = 10

wnc = WaveNetClassifier((23962,), (10,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 23962)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 23962, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 23962, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 23962, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [154]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 32, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
10/10 [==============================] - 9s 862ms/step - loss: 2.3025 - acc: 0.1000

Epoch 00001: loss improved from inf to 2.30248, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
10/10 [==============================] - 1s 76ms/step - loss: 2.3022 - acc: 0.1000

Epoch 00002: loss improved from 2.30248 to 2.30217, saving model to results3/saved_wavenet_clasifier.h5
Epoch 3/200
10/10 [==============================] - 1s 73ms/step - loss: 2.2996 - acc: 0.1000

Epoch 00003: loss improved from 2.30217 to 2.29960, saving model to results3/saved_wavenet_clasifier.h5
Epoch 4/200
10/10 [==============================] - 1s 72ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00004: loss did not improve from 2.29960
Epoch 5/200
10/10 [==============================] - 1s 72ms/step - loss: 2.2900 - acc: 0.2000

Epoch 00005: loss improved from 2.29960 to 2.29002, saving model to results3/saved_wavenet_clasifier.h5
Epoch 6/200
10/10 [==============================] - 1s 72ms


Epoch 00044: loss did not improve from 0.27330
Epoch 45/200
10/10 [==============================] - 1s 76ms/step - loss: 1.1061 - acc: 0.6000

Epoch 00045: loss did not improve from 0.27330
Epoch 46/200
10/10 [==============================] - 1s 76ms/step - loss: 1.3860 - acc: 0.3000

Epoch 00046: loss did not improve from 0.27330
Epoch 47/200
10/10 [==============================] - 1s 76ms/step - loss: 1.5713 - acc: 0.4000

Epoch 00047: loss did not improve from 0.27330
Epoch 48/200
10/10 [==============================] - 1s 76ms/step - loss: 1.4892 - acc: 0.4000

Epoch 00048: loss did not improve from 0.27330
Epoch 49/200
10/10 [==============================] - 1s 76ms/step - loss: 1.3818 - acc: 0.4000

Epoch 00049: loss did not improve from 0.27330
Epoch 50/200
10/10 [==============================] - 1s 76ms/step - loss: 1.2392 - acc: 0.4000

Epoch 00050: loss did not improve from 0.27330
Epoch 51/200
10/10 [==============================] - 1s 76ms/step - loss: 1.1535 - acc:


Epoch 00092: loss improved from 0.10953 to 0.10345, saving model to results3/saved_wavenet_clasifier.h5
Epoch 93/200
10/10 [==============================] - 1s 76ms/step - loss: 0.0983 - acc: 1.0000

Epoch 00093: loss improved from 0.10345 to 0.09828, saving model to results3/saved_wavenet_clasifier.h5
Epoch 94/200
10/10 [==============================] - 1s 76ms/step - loss: 0.0952 - acc: 1.0000

Epoch 00094: loss improved from 0.09828 to 0.09517, saving model to results3/saved_wavenet_clasifier.h5
Epoch 95/200
10/10 [==============================] - 1s 75ms/step - loss: 0.1114 - acc: 0.9000

Epoch 00095: loss did not improve from 0.09517
Epoch 96/200
10/10 [==============================] - 1s 74ms/step - loss: 0.1144 - acc: 0.9000

Epoch 00096: loss did not improve from 0.09517
Epoch 97/200
10/10 [==============================] - 1s 76ms/step - loss: 0.1015 - acc: 0.9000

Epoch 00097: loss did not improve from 0.09517
Epoch 98/200
10/10 [==============================] - 1s 76ms

KeyboardInterrupt: 

In [159]:
# 1 sample for 1 user
# no_users = 20

wnc = WaveNetClassifier((18330,), (20,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 18330)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 18330, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 18330, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 18330, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [160]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 32, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
20/20 [==============================] - 9s 438ms/step - loss: 2.9955 - acc: 0.0500

Epoch 00001: loss improved from inf to 2.99554, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
20/20 [==============================] - 1s 56ms/step - loss: 2.9962 - acc: 0.0500

Epoch 00002: loss did not improve from 2.99554
Epoch 3/200
20/20 [==============================] - 1s 54ms/step - loss: 2.9927 - acc: 0.0500

Epoch 00003: loss improved from 2.99554 to 2.99274, saving model to results3/saved_wavenet_clasifier.h5
Epoch 4/200
20/20 [==============================] - 1s 54ms/step - loss: 2.9888 - acc: 0.0500

Epoch 00004: loss improved from 2.99274 to 2.98877, saving model to results3/saved_wavenet_clasifier.h5
Epoch 5/200
20/20 [==============================] - 1s 54ms/step - loss: 2.9783 - acc: 0.0500

Epoch 00005: loss improved from 2.98877 to 2.97829, saving model to results3/saved_wavenet_clasifier.h5
Epoch 6/200
20/20 [==============================] - 1s 54ms

Epoch 44/200
20/20 [==============================] - 1s 54ms/step - loss: 0.2339 - acc: 0.9000

Epoch 00044: loss did not improve from 0.18597
Epoch 45/200
20/20 [==============================] - 1s 54ms/step - loss: 0.3065 - acc: 0.8500

Epoch 00045: loss did not improve from 0.18597
Epoch 46/200
20/20 [==============================] - 1s 54ms/step - loss: 0.1314 - acc: 0.9500

Epoch 00046: loss improved from 0.18597 to 0.13136, saving model to results3/saved_wavenet_clasifier.h5
Epoch 47/200
20/20 [==============================] - 1s 54ms/step - loss: 0.2093 - acc: 0.9500

Epoch 00047: loss did not improve from 0.13136
Epoch 48/200
20/20 [==============================] - 1s 54ms/step - loss: 0.1001 - acc: 1.0000

Epoch 00048: loss improved from 0.13136 to 0.10015, saving model to results3/saved_wavenet_clasifier.h5
Epoch 49/200
20/20 [==============================] - 1s 54ms/step - loss: 0.1780 - acc: 0.9000

Epoch 00049: loss did not improve from 0.10015
Epoch 50/200
20/20 [==


Epoch 00096: loss did not improve from 0.02130
Epoch 97/200
20/20 [==============================] - 1s 54ms/step - loss: 0.0183 - acc: 1.0000

Epoch 00097: loss improved from 0.02130 to 0.01835, saving model to results3/saved_wavenet_clasifier.h5
Epoch 98/200
20/20 [==============================] - 1s 54ms/step - loss: 0.0138 - acc: 1.0000

Epoch 00098: loss improved from 0.01835 to 0.01381, saving model to results3/saved_wavenet_clasifier.h5
Epoch 99/200
20/20 [==============================] - 1s 54ms/step - loss: 0.0139 - acc: 1.0000

Epoch 00099: loss did not improve from 0.01381
Epoch 100/200
20/20 [==============================] - 1s 54ms/step - loss: 0.0147 - acc: 1.0000

Epoch 00100: loss did not improve from 0.01381
Epoch 101/200
20/20 [==============================] - 1s 54ms/step - loss: 0.0116 - acc: 1.0000

Epoch 00101: loss improved from 0.01381 to 0.01163, saving model to results3/saved_wavenet_clasifier.h5
Epoch 102/200
20/20 [==============================] - 1s 5

Epoch 139/200
20/20 [==============================] - 1s 54ms/step - loss: 9.7220e-04 - acc: 1.0000

Epoch 00139: loss improved from 0.00100 to 0.00097, saving model to results3/saved_wavenet_clasifier.h5
Epoch 140/200
20/20 [==============================] - 1s 54ms/step - loss: 9.4536e-04 - acc: 1.0000

Epoch 00140: loss improved from 0.00097 to 0.00095, saving model to results3/saved_wavenet_clasifier.h5
Epoch 141/200
20/20 [==============================] - 1s 54ms/step - loss: 9.2363e-04 - acc: 1.0000

Epoch 00141: loss improved from 0.00095 to 0.00092, saving model to results3/saved_wavenet_clasifier.h5
Epoch 142/200
20/20 [==============================] - 1s 54ms/step - loss: 9.0555e-04 - acc: 1.0000

Epoch 00142: loss improved from 0.00092 to 0.00091, saving model to results3/saved_wavenet_clasifier.h5
Epoch 143/200
20/20 [==============================] - 1s 54ms/step - loss: 8.8881e-04 - acc: 1.0000

Epoch 00143: loss improved from 0.00091 to 0.00089, saving model to result

Epoch 179/200
20/20 [==============================] - 1s 54ms/step - loss: 4.8602e-04 - acc: 1.0000

Epoch 00179: loss improved from 0.00049 to 0.00049, saving model to results3/saved_wavenet_clasifier.h5
Epoch 180/200
20/20 [==============================] - 1s 54ms/step - loss: 4.7981e-04 - acc: 1.0000

Epoch 00180: loss improved from 0.00049 to 0.00048, saving model to results3/saved_wavenet_clasifier.h5
Epoch 181/200
20/20 [==============================] - 1s 54ms/step - loss: 4.7371e-04 - acc: 1.0000

Epoch 00181: loss improved from 0.00048 to 0.00047, saving model to results3/saved_wavenet_clasifier.h5
Epoch 182/200
20/20 [==============================] - 1s 54ms/step - loss: 4.6774e-04 - acc: 1.0000

Epoch 00182: loss improved from 0.00047 to 0.00047, saving model to results3/saved_wavenet_clasifier.h5
Epoch 183/200
20/20 [==============================] - 1s 54ms/step - loss: 4.6192e-04 - acc: 1.0000

Epoch 00183: loss improved from 0.00047 to 0.00046, saving model to result

In [164]:
# 1 sample for 1 user
# no_users = 50

wnc = WaveNetClassifier((18330,), (50,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 18330)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 18330, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 18330, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 18330, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [167]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 20, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
50/50 [==============================] - 11s 217ms/step - loss: 3.9160 - acc: 0.0000e+00

Epoch 00001: loss improved from inf to 3.91604, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
50/50 [==============================] - 3s 58ms/step - loss: 3.9130 - acc: 0.0200

Epoch 00002: loss improved from 3.91604 to 3.91301, saving model to results3/saved_wavenet_clasifier.h5
Epoch 3/200
50/50 [==============================] - 3s 58ms/step - loss: 3.9115 - acc: 0.0200

Epoch 00003: loss improved from 3.91301 to 3.91152, saving model to results3/saved_wavenet_clasifier.h5
Epoch 4/200
50/50 [==============================] - 3s 58ms/step - loss: 3.9144 - acc: 0.0200

Epoch 00004: loss did not improve from 3.91152
Epoch 5/200
50/50 [==============================] - 3s 58ms/step - loss: 3.8914 - acc: 0.0200

Epoch 00005: loss improved from 3.91152 to 3.89143, saving model to results3/saved_wavenet_clasifier.h5
Epoch 6/200
50/50 [==============================] - 3s

Epoch 46/200
50/50 [==============================] - 3s 58ms/step - loss: 1.5141 - acc: 0.6000

Epoch 00046: loss improved from 1.57615 to 1.51405, saving model to results3/saved_wavenet_clasifier.h5
Epoch 47/200
50/50 [==============================] - 3s 58ms/step - loss: 1.3515 - acc: 0.6000

Epoch 00047: loss improved from 1.51405 to 1.35154, saving model to results3/saved_wavenet_clasifier.h5
Epoch 48/200
50/50 [==============================] - 3s 58ms/step - loss: 1.2754 - acc: 0.6200

Epoch 00048: loss improved from 1.35154 to 1.27537, saving model to results3/saved_wavenet_clasifier.h5
Epoch 49/200
50/50 [==============================] - 3s 58ms/step - loss: 1.2700 - acc: 0.6800

Epoch 00049: loss improved from 1.27537 to 1.27005, saving model to results3/saved_wavenet_clasifier.h5
Epoch 50/200
50/50 [==============================] - 3s 58ms/step - loss: 1.2258 - acc: 0.6400

Epoch 00050: loss improved from 1.27005 to 1.22584, saving model to results3/saved_wavenet_clasifie

Epoch 94/200
50/50 [==============================] - 3s 56ms/step - loss: 0.6458 - acc: 0.9600

Epoch 00094: loss improved from 0.64604 to 0.64580, saving model to results3/saved_wavenet_clasifier.h5
Epoch 95/200
50/50 [==============================] - 3s 55ms/step - loss: 0.6457 - acc: 0.9600

Epoch 00095: loss improved from 0.64580 to 0.64568, saving model to results3/saved_wavenet_clasifier.h5
Epoch 96/200
50/50 [==============================] - 3s 55ms/step - loss: 0.6456 - acc: 0.9600

Epoch 00096: loss improved from 0.64568 to 0.64557, saving model to results3/saved_wavenet_clasifier.h5
Epoch 97/200
50/50 [==============================] - 3s 55ms/step - loss: 0.6455 - acc: 0.9600

Epoch 00097: loss improved from 0.64557 to 0.64550, saving model to results3/saved_wavenet_clasifier.h5
Epoch 98/200
50/50 [==============================] - 3s 55ms/step - loss: 0.6454 - acc: 0.9600

Epoch 00098: loss improved from 0.64550 to 0.64544, saving model to results3/saved_wavenet_clasifie

KeyboardInterrupt: 

In [171]:
# 1 sample for 1 user
# no_users = 100

wnc = WaveNetClassifier((18330,), (100,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 18330)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 18330, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 18330, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 18330, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [172]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 20, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
100/100 [==============================] - 13s 125ms/step - loss: 4.6101 - acc: 0.0000e+00

Epoch 00001: loss improved from inf to 4.61014, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
100/100 [==============================] - 6s 57ms/step - loss: 4.6058 - acc: 0.0000e+00

Epoch 00002: loss improved from 4.61014 to 4.60575, saving model to results3/saved_wavenet_clasifier.h5
Epoch 3/200
100/100 [==============================] - 6s 57ms/step - loss: 4.6055 - acc: 0.0100

Epoch 00003: loss improved from 4.60575 to 4.60550, saving model to results3/saved_wavenet_clasifier.h5
Epoch 4/200
100/100 [==============================] - 6s 55ms/step - loss: 4.6054 - acc: 0.0100

Epoch 00004: loss improved from 4.60550 to 4.60543, saving model to results3/saved_wavenet_clasifier.h5
Epoch 5/200
100/100 [==============================] - 6s 55ms/step - loss: 4.6054 - acc: 0.0100

Epoch 00005: loss improved from 4.60543 to 4.60542, saving model to results3/saved_waven

100/100 [==============================] - 6s 58ms/step - loss: 4.6054 - acc: 0.0000e+00

Epoch 00050: loss did not improve from 4.60536
Epoch 51/200
100/100 [==============================] - 6s 58ms/step - loss: 4.6054 - acc: 0.0100

Epoch 00051: loss did not improve from 4.60536
Epoch 52/200
100/100 [==============================] - 6s 58ms/step - loss: 4.6054 - acc: 0.0000e+00

Epoch 00052: loss did not improve from 4.60536
Epoch 53/200
100/100 [==============================] - 6s 58ms/step - loss: 4.6054 - acc: 0.0100

Epoch 00053: loss did not improve from 4.60536
Epoch 54/200
100/100 [==============================] - 6s 58ms/step - loss: 4.6054 - acc: 0.0000e+00

Epoch 00054: loss did not improve from 4.60536
Epoch 55/200
100/100 [==============================] - 6s 58ms/step - loss: 4.6054 - acc: 0.0000e+00

Epoch 00055: loss did not improve from 4.60536
Epoch 56/200
100/100 [==============================] - 6s 58ms/step - loss: 4.6054 - acc: 0.0100

Epoch 00056: loss impr

KeyboardInterrupt: 

In [178]:
# 2 samples for 1 user
# no_users = 10

wnc = WaveNetClassifier((23962,), (10,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 23962)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 23962, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 23962, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 23962, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [182]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 16, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
20/20 [==============================] - 12s 617ms/step - loss: 2.3040 - acc: 0.1000

Epoch 00001: loss improved from inf to 2.30398, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
20/20 [==============================] - 1s 74ms/step - loss: 2.3121 - acc: 0.1000

Epoch 00002: loss did not improve from 2.30398
Epoch 3/200
20/20 [==============================] - 1s 74ms/step - loss: 2.3032 - acc: 0.1000

Epoch 00003: loss improved from 2.30398 to 2.30319, saving model to results3/saved_wavenet_clasifier.h5
Epoch 4/200
20/20 [==============================] - 1s 72ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00004: loss improved from 2.30319 to 2.30275, saving model to results3/saved_wavenet_clasifier.h5
Epoch 5/200
20/20 [==============================] - 1s 73ms/step - loss: 2.3028 - acc: 0.1000

Epoch 00005: loss did not improve from 2.30275
Epoch 6/200
20/20 [==============================] - 1s 73ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00006: loss im

20/20 [==============================] - 1s 74ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00055: loss did not improve from 2.30255
Epoch 56/200
20/20 [==============================] - 1s 72ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00056: loss did not improve from 2.30255
Epoch 57/200
20/20 [==============================] - 1s 74ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00057: loss did not improve from 2.30255
Epoch 58/200
20/20 [==============================] - 2s 77ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00058: loss did not improve from 2.30255
Epoch 59/200
20/20 [==============================] - 2s 77ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00059: loss did not improve from 2.30255
Epoch 60/200
20/20 [==============================] - 2s 77ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00060: loss did not improve from 2.30255
Epoch 61/200
20/20 [==============================] - 2s 78ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00061: loss did not improve from 2.30255
Epoch

20/20 [==============================] - 1s 74ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00112: loss did not improve from 2.30255
Epoch 113/200
20/20 [==============================] - 1s 75ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00113: loss did not improve from 2.30255
Epoch 114/200
20/20 [==============================] - 2s 76ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00114: loss did not improve from 2.30255
Epoch 115/200
20/20 [==============================] - 1s 73ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00115: loss did not improve from 2.30255
Epoch 116/200
20/20 [==============================] - 1s 73ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00116: loss did not improve from 2.30255
Epoch 117/200
20/20 [==============================] - 1s 73ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00117: loss did not improve from 2.30255
Epoch 118/200
20/20 [==============================] - 1s 74ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00118: loss did not improve from 2.30255

20/20 [==============================] - 1s 73ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00168: loss did not improve from 2.30255
Epoch 169/200
20/20 [==============================] - 1s 73ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00169: loss did not improve from 2.30255
Epoch 170/200
20/20 [==============================] - 1s 73ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00170: loss did not improve from 2.30255
Epoch 171/200
20/20 [==============================] - 2s 76ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00171: loss did not improve from 2.30255
Epoch 172/200
20/20 [==============================] - 2s 76ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00172: loss did not improve from 2.30255
Epoch 173/200
20/20 [==============================] - 2s 75ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00173: loss did not improve from 2.30255
Epoch 174/200
20/20 [==============================] - 1s 73ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00174: loss did not improve from 2.30255

In [183]:
# # 2 samples for 1 user
# no_users = 10
# kernel_size = 5

wnc = WaveNetClassifier((23962,), (10,), kernel_size = 5, dilation_depth = 9, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 23962)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 23962, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 23962, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_5_tanh (Conv1D)    (None, 23962, 40)    8040        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [186]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 1, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200


ResourceExhaustedError: OOM when allocating tensor with shape[1953125,5,40] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node dilated_conv_1953125_tanh/convolution/SpaceToBatchND}} = SpaceToBatchND[T=DT_FLOAT, Tblock_shape=DT_INT32, Tpaddings=DT_INT32, _class=["loc:@train...hToSpaceND"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](dilated_conv_1953125_tanh/Pad, dilated_conv_1953125_tanh/convolution/SpaceToBatchND/block_shape, dilated_conv_1953125_tanh/convolution/SpaceToBatchND/paddings)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics_21/acc/Mean/_15275}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_5362_metrics_21/acc/Mean", tensor_type=DT_FLOAT, _devi

In [190]:
# # 2 samples for 1 user
# no_users = 10
# dilation_depth = 5

wnc = WaveNetClassifier((23962,), (10,), kernel_size = 2, dilation_depth = 5, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 23962)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 23962, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 23962, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 23962, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [191]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 16, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
20/20 [==============================] - 11s 531ms/step - loss: 2.3029 - acc: 0.1000

Epoch 00001: loss improved from inf to 2.30286, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
20/20 [==============================] - 1s 50ms/step - loss: 2.3054 - acc: 0.0500

Epoch 00002: loss did not improve from 2.30286
Epoch 3/200
20/20 [==============================] - 1s 48ms/step - loss: 2.3030 - acc: 0.1000

Epoch 00003: loss did not improve from 2.30286
Epoch 4/200
20/20 [==============================] - 1s 48ms/step - loss: 2.2996 - acc: 0.1000

Epoch 00004: loss improved from 2.30286 to 2.29961, saving model to results3/saved_wavenet_clasifier.h5
Epoch 5/200
20/20 [==============================] - 1s 49ms/step - loss: 2.2920 - acc: 0.1000

Epoch 00005: loss improved from 2.29961 to 2.29203, saving model to results3/saved_wavenet_clasifier.h5
Epoch 6/200
20/20 [==============================] - 1s 48ms/step - loss: 2.2875 - acc: 0.1000

Epoch 00006: loss im

20/20 [==============================] - 1s 50ms/step - loss: 1.7925 - acc: 0.3500

Epoch 00050: loss improved from 1.83774 to 1.79248, saving model to results3/saved_wavenet_clasifier.h5
Epoch 51/200
20/20 [==============================] - 1s 50ms/step - loss: 1.8704 - acc: 0.3000

Epoch 00051: loss did not improve from 1.79248
Epoch 52/200
20/20 [==============================] - 1s 50ms/step - loss: 1.7926 - acc: 0.3500

Epoch 00052: loss did not improve from 1.79248
Epoch 53/200
20/20 [==============================] - 1s 50ms/step - loss: 1.8818 - acc: 0.3500

Epoch 00053: loss did not improve from 1.79248
Epoch 54/200
20/20 [==============================] - 1s 50ms/step - loss: 1.9484 - acc: 0.3000

Epoch 00054: loss did not improve from 1.79248
Epoch 55/200
20/20 [==============================] - 1s 50ms/step - loss: 1.9579 - acc: 0.3000

Epoch 00055: loss did not improve from 1.79248
Epoch 56/200
20/20 [==============================] - 1s 50ms/step - loss: 1.9220 - acc: 0.3

20/20 [==============================] - 1s 50ms/step - loss: 3.2578 - acc: 0.3000

Epoch 00096: loss did not improve from 0.85013
Epoch 97/200
20/20 [==============================] - 1s 50ms/step - loss: 3.6797 - acc: 0.2000

Epoch 00097: loss did not improve from 0.85013
Epoch 98/200
20/20 [==============================] - 1s 50ms/step - loss: 2.5910 - acc: 0.4000

Epoch 00098: loss did not improve from 0.85013
Epoch 99/200
20/20 [==============================] - 1s 50ms/step - loss: 2.4718 - acc: 0.4000

Epoch 00099: loss did not improve from 0.85013
Epoch 100/200
20/20 [==============================] - 1s 50ms/step - loss: 2.3765 - acc: 0.4500

Epoch 00100: loss did not improve from 0.85013
Epoch 101/200
20/20 [==============================] - 1s 50ms/step - loss: 2.2610 - acc: 0.4000

Epoch 00101: loss did not improve from 0.85013
Epoch 102/200
20/20 [==============================] - 1s 50ms/step - loss: 2.4596 - acc: 0.3000

Epoch 00102: loss did not improve from 0.85013
Ep

20/20 [==============================] - 1s 48ms/step - loss: 0.5590 - acc: 0.7500

Epoch 00145: loss improved from 0.56783 to 0.55900, saving model to results3/saved_wavenet_clasifier.h5
Epoch 146/200
20/20 [==============================] - 1s 48ms/step - loss: 0.5540 - acc: 0.7500

Epoch 00146: loss improved from 0.55900 to 0.55405, saving model to results3/saved_wavenet_clasifier.h5
Epoch 147/200
20/20 [==============================] - 1s 48ms/step - loss: 0.5448 - acc: 0.7500

Epoch 00147: loss improved from 0.55405 to 0.54482, saving model to results3/saved_wavenet_clasifier.h5
Epoch 148/200
20/20 [==============================] - 1s 48ms/step - loss: 0.5304 - acc: 0.7500

Epoch 00148: loss improved from 0.54482 to 0.53042, saving model to results3/saved_wavenet_clasifier.h5
Epoch 149/200
20/20 [==============================] - 1s 48ms/step - loss: 0.5360 - acc: 0.7500

Epoch 00149: loss did not improve from 0.53042
Epoch 150/200
20/20 [==============================] - 1s 48m

Epoch 186/200
20/20 [==============================] - 1s 51ms/step - loss: 0.3213 - acc: 0.8500

Epoch 00186: loss improved from 0.32629 to 0.32133, saving model to results3/saved_wavenet_clasifier.h5
Epoch 187/200
20/20 [==============================] - 1s 50ms/step - loss: 0.3175 - acc: 0.8500

Epoch 00187: loss improved from 0.32133 to 0.31751, saving model to results3/saved_wavenet_clasifier.h5
Epoch 188/200
20/20 [==============================] - 1s 51ms/step - loss: 0.3092 - acc: 0.8500

Epoch 00188: loss improved from 0.31751 to 0.30917, saving model to results3/saved_wavenet_clasifier.h5
Epoch 189/200
20/20 [==============================] - 1s 51ms/step - loss: 0.3065 - acc: 0.8500

Epoch 00189: loss improved from 0.30917 to 0.30654, saving model to results3/saved_wavenet_clasifier.h5
Epoch 190/200
20/20 [==============================] - 1s 51ms/step - loss: 0.3035 - acc: 0.8500

Epoch 00190: loss improved from 0.30654 to 0.30349, saving model to results3/saved_wavenet_cla

In [203]:
# # 2 samples for 1 user
# no_users = 10
# dilation_depth = 2

wnc = WaveNetClassifier((23962,), (10,), kernel_size = 2, dilation_depth = 2, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 23962)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 23962, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 23962, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 23962, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [204]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 16, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
20/20 [==============================] - 10s 509ms/step - loss: 2.3029 - acc: 0.0500

Epoch 00001: loss improved from inf to 2.30293, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
20/20 [==============================] - 1s 34ms/step - loss: 2.3029 - acc: 0.1000

Epoch 00002: loss improved from 2.30293 to 2.30290, saving model to results3/saved_wavenet_clasifier.h5
Epoch 3/200
20/20 [==============================] - 1s 31ms/step - loss: 2.3036 - acc: 0.1000

Epoch 00003: loss did not improve from 2.30290
Epoch 4/200
20/20 [==============================] - 1s 31ms/step - loss: 2.3037 - acc: 0.1000

Epoch 00004: loss did not improve from 2.30290
Epoch 5/200
20/20 [==============================] - 1s 31ms/step - loss: 2.3026 - acc: 0.1000

Epoch 00005: loss improved from 2.30290 to 2.30261, saving model to results3/saved_wavenet_clasifier.h5
Epoch 6/200
20/20 [==============================] - 1s 31ms/step - loss: 2.3027 - acc: 0.1000

Epoch 00006: loss di

Epoch 48/200
20/20 [==============================] - 1s 29ms/step - loss: 2.4372 - acc: 0.0500

Epoch 00048: loss did not improve from 2.02985
Epoch 49/200
20/20 [==============================] - 1s 30ms/step - loss: 2.0322 - acc: 0.1500

Epoch 00049: loss did not improve from 2.02985
Epoch 50/200
20/20 [==============================] - 1s 30ms/step - loss: 1.9990 - acc: 0.1500

Epoch 00050: loss improved from 2.02985 to 1.99898, saving model to results3/saved_wavenet_clasifier.h5
Epoch 51/200
20/20 [==============================] - 1s 30ms/step - loss: 2.0438 - acc: 0.1500

Epoch 00051: loss did not improve from 1.99898
Epoch 52/200
20/20 [==============================] - 1s 30ms/step - loss: 2.0188 - acc: 0.1500

Epoch 00052: loss did not improve from 1.99898
Epoch 53/200
20/20 [==============================] - 1s 31ms/step - loss: 2.0096 - acc: 0.2000

Epoch 00053: loss did not improve from 1.99898
Epoch 54/200
20/20 [==============================] - 1s 30ms/step - loss: 2.00

Epoch 93/200
20/20 [==============================] - 1s 29ms/step - loss: 1.1494 - acc: 0.6000

Epoch 00093: loss improved from 1.20399 to 1.14938, saving model to results3/saved_wavenet_clasifier.h5
Epoch 94/200
20/20 [==============================] - 1s 30ms/step - loss: 1.1544 - acc: 0.6000

Epoch 00094: loss did not improve from 1.14938
Epoch 95/200
20/20 [==============================] - 1s 30ms/step - loss: 1.1412 - acc: 0.6500

Epoch 00095: loss improved from 1.14938 to 1.14125, saving model to results3/saved_wavenet_clasifier.h5
Epoch 96/200
20/20 [==============================] - 1s 31ms/step - loss: 1.1128 - acc: 0.5500

Epoch 00096: loss improved from 1.14125 to 1.11281, saving model to results3/saved_wavenet_clasifier.h5
Epoch 97/200
20/20 [==============================] - 1s 32ms/step - loss: 1.1001 - acc: 0.4500

Epoch 00097: loss improved from 1.11281 to 1.10008, saving model to results3/saved_wavenet_clasifier.h5
Epoch 98/200
20/20 [==============================] 

20/20 [==============================] - 1s 30ms/step - loss: 0.2648 - acc: 0.9000

Epoch 00139: loss improved from 0.29258 to 0.26483, saving model to results3/saved_wavenet_clasifier.h5
Epoch 140/200
20/20 [==============================] - 1s 30ms/step - loss: 0.7555 - acc: 0.7500

Epoch 00140: loss did not improve from 0.26483
Epoch 141/200
20/20 [==============================] - 1s 31ms/step - loss: 0.2649 - acc: 0.8500

Epoch 00141: loss did not improve from 0.26483
Epoch 142/200
20/20 [==============================] - 1s 30ms/step - loss: 0.5734 - acc: 0.7500

Epoch 00142: loss did not improve from 0.26483
Epoch 143/200
20/20 [==============================] - 1s 30ms/step - loss: 0.4622 - acc: 0.8000

Epoch 00143: loss did not improve from 0.26483
Epoch 144/200
20/20 [==============================] - 1s 30ms/step - loss: 0.2712 - acc: 0.9000

Epoch 00144: loss did not improve from 0.26483
Epoch 145/200
20/20 [==============================] - 1s 30ms/step - loss: 0.2706 - ac

KeyboardInterrupt: 

In [9]:
# # 2 samples for 1 user
# no_users = 400
# dilation_depth = 2

wnc = WaveNetClassifier((18330,), (100,), kernel_size = 2, dilation_depth = 2, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 18330)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 18330, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 18330, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 18330, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [10]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 16, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
200/200 [==============================] - 15s 76ms/step - loss: 4.6122 - acc: 0.0000e+00

Epoch 00001: loss improved from inf to 4.61221, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
200/200 [==============================] - 4s 22ms/step - loss: 4.6057 - acc: 0.0100

Epoch 00002: loss improved from 4.61221 to 4.60571, saving model to results3/saved_wavenet_clasifier.h5
Epoch 3/200
200/200 [==============================] - 4s 22ms/step - loss: 4.6058 - acc: 0.0000e+00

Epoch 00003: loss did not improve from 4.60571
Epoch 4/200
200/200 [==============================] - 4s 22ms/step - loss: 4.6057 - acc: 0.0100

Epoch 00004: loss improved from 4.60571 to 4.60568, saving model to results3/saved_wavenet_clasifier.h5
Epoch 5/200
200/200 [==============================] - 4s 22ms/step - loss: 4.6057 - acc: 0.0000e+00

Epoch 00005: loss improved from 4.60568 to 4.60566, saving model to results3/saved_wavenet_clasifier.h5
Epoch 6/200
200/200 [=================

200/200 [==============================] - 5s 23ms/step - loss: 1.4781 - acc: 0.6150

Epoch 00044: loss improved from 1.56090 to 1.47809, saving model to results3/saved_wavenet_clasifier.h5
Epoch 45/200
200/200 [==============================] - 5s 23ms/step - loss: 1.3900 - acc: 0.6200

Epoch 00045: loss improved from 1.47809 to 1.38995, saving model to results3/saved_wavenet_clasifier.h5
Epoch 46/200
200/200 [==============================] - 5s 23ms/step - loss: 1.4253 - acc: 0.6000

Epoch 00046: loss did not improve from 1.38995
Epoch 47/200
200/200 [==============================] - 5s 23ms/step - loss: 1.4739 - acc: 0.6150

Epoch 00047: loss did not improve from 1.38995
Epoch 48/200
200/200 [==============================] - 5s 23ms/step - loss: 1.2394 - acc: 0.6350

Epoch 00048: loss improved from 1.38995 to 1.23945, saving model to results3/saved_wavenet_clasifier.h5
Epoch 49/200
200/200 [==============================] - 5s 23ms/step - loss: 1.1756 - acc: 0.6400

Epoch 00049: 

KeyboardInterrupt: 

In [14]:
# # 9 samples for 1 user
# no_users = 100
# dilation_depth = 2

wnc = WaveNetClassifier((18330,), (100,), kernel_size = 2, dilation_depth = 2, n_filters = 40, task = 'classification')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_input (InputLayer)     (None, 18330)        0                                            
__________________________________________________________________________________________________
reshaped_input (Reshape)        (None, 18330, 1)     0           original_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 18330, 40)    120         reshaped_input[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 18330, 40)    3240        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_co

In [15]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 16, optimizer='adam', save=True, save_dir='results3/')

Epoch 1/200
900/900 [==============================] - 22s 24ms/step - loss: 4.6080 - acc: 0.0044

Epoch 00001: loss improved from inf to 4.60805, saving model to results3/saved_wavenet_clasifier.h5
Epoch 2/200
900/900 [==============================] - 20s 22ms/step - loss: 4.6061 - acc: 0.0033

Epoch 00002: loss improved from 4.60805 to 4.60614, saving model to results3/saved_wavenet_clasifier.h5
Epoch 3/200
900/900 [==============================] - 20s 22ms/step - loss: 4.6060 - acc: 0.0067

Epoch 00003: loss improved from 4.60614 to 4.60601, saving model to results3/saved_wavenet_clasifier.h5
Epoch 4/200
900/900 [==============================] - 20s 22ms/step - loss: 4.6061 - acc: 0.0033

Epoch 00004: loss did not improve from 4.60601
Epoch 5/200
900/900 [==============================] - 20s 22ms/step - loss: 4.6060 - acc: 0.0044

Epoch 00005: loss improved from 4.60601 to 4.60597, saving model to results3/saved_wavenet_clasifier.h5
Epoch 6/200
900/900 [=========================

KeyboardInterrupt: 